# NSIDC EGI Interface

- https://github.com/creare-com/podpac/issues/237
- https://developer.earthdata.nasa.gov/sdps/programmatic-access-docs#curl
- https://nsidc.org/support/how/how-do-i-programmatically-request-data-services

In [5]:
import requests
import xml.etree.ElementTree

## Obtain Token

https://wiki.earthdata.nasa.gov/display/CMR/Creating+a+Token+Common

In [6]:
username = "username"
password = "password"
ip = 'ip'
url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'

In [7]:
request = """
<token>
    <username>{username}</username>
    <password>{password}</password>
    <client_id>podpac</client_id>
    <user_ip_address>{ip}</user_ip_address>
</token>
""".format(username=username, password=password,ip=ip)
headers = {"Content-Type": "application/xml"}
r = requests.post(url, data=request, headers=headers)

In [8]:
tree = xml.etree.ElementTree.fromstring(r.text)
token = [element.text for element in tree.findall('id')][0]

## Get Data

In [42]:
base_url = "https://n5eil01u.ecs.nsidc.org/egi/request"
product = "SPL3SMA"
version = "003"
start_time = "2015-03-30"
end_time = "2015-06-12"
data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"
bbox = "-70,42,-69,43"
response_format = "HDF-EOS5"
token = token  # from above

In [43]:
url = "{base_url}?short_name={product}&version={version}".format(base_url=base_url, product=product, version=version,) + \
      "&time={start_time},{end_time}&Subset_Data_Layers={data_layers}".format(start_time=start_time, end_time=end_time, data_layers=data_layers, ) + \
      "&Bbox={bbox}&format={response_format}&token={token}".format(bbox=bbox, response_format=response_format, token=token)

r = requests.get(url)

In [44]:
with open('output.h5', 'wb') as f:
    f.write(r.content)

## Look at the Data